This notebook is for demonstrating how to use a trained PM2.5 prediction model to make predictions on a target area.

# Imports

In [172]:
%load_ext autoreload 
%autoreload 2

import geopandas as gpd
import re

import joblib
import numpy as np
import pandas as pd

import sys
sys.path.append("../../")

from src.config import settings
from src.data_processing import feature_collection_pipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [162]:

# CSV file for target roll-out location
CENTROIDS_PATH = "data/chiang_mai_tile_centroids.csv"
# ID Column in the target CSV file
ID_COL = "id"

# Desired date range
START_DATE = "2019-03-01"
END_DATE = "2019-03-30"
# Model being used was trained using 1km x 1km bounding box
BBOX_SIZE_KM = 1

# Path to the model to be used for prediction
MODEL_PATH = settings.DATA_DIR / "latest_model.pkl"

# Path to the population raster (In our study, we're using the Thai 2020 HRSL raster)
HRSL_TIF = settings.DATA_DIR / "tha_general_2020.tif"

# Where to save the model predictions of  daily PM2.5 levels
OUTPUT_PREDICTIONS_FILE = settings.DATA_DIR / "chiang_mai_march2019_predictions.csv"

# Load Area of Interest

In [163]:
def extract_lon_lat(wkt_string_point):
    regex = r'[0-9-\.]+'
    parsed_geom = re.findall(regex, wkt_string_point)
    parsed_geom = [float(i) for i in parsed_geom]
    assert len(parsed_geom) == 2
    return parsed_geom[0], parsed_geom[1]


def load_centroids(centroids_path):
    locations_gdf = gpd.read_file(centroids_path)
    all_lon_lat = locations_gdf["WKT"].apply(lambda x: extract_lon_lat(x)).tolist()
    all_lon, all_lat = zip(*all_lon_lat)

    locations_gdf["longitude"] = all_lon
    locations_gdf["latitude"] = all_lat 

    locations_df = pd.DataFrame(locations_gdf)
    locations_df.drop(["geometry", "WKT"], axis=1, inplace=True)

    return locations_df
    

In [164]:
locations_df = load_centroids(CENTROIDS_PATH)[:]
len(locations_df)

206

In [165]:
locations_df.columns

Index(['id', 'left', 'top', 'right', 'bottom', 'Shape_Leng', 'Shape_Area',
       'ADM3_EN', 'ADM3_TH', 'ADM3_PCODE', 'ADM3_REF', 'ADM3ALT1EN',
       'ADM3ALT2EN', 'ADM3ALT1TH', 'ADM3ALT2TH', 'ADM2_EN', 'ADM2_TH',
       'ADM2_PCODE', 'ADM1_EN', 'ADM1_TH', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_TH',
       'ADM0_PCODE', 'date', 'validOn', 'validTo', 'longitude', 'latitude'],
      dtype='object')

# Predict

In [166]:
# Create base DF from the locations (collect the necessary features)
base_df = feature_collection_pipeline.collect_features_for_locations(
    locations_df=locations_df,
    start_date=START_DATE,
    end_date=END_DATE,
    id_col=ID_COL,
    hrsl_tif=HRSL_TIF,
    bbox_size_km=BBOX_SIZE_KM,
    # Customized the list of GEE datasets because the latest model doesn't use MAIAC
    gee_datasets=[
        feature_collection_pipeline.S5P_AAI_CONFIG,
        feature_collection_pipeline.CAMS_AOD_CONFIG,
        feature_collection_pipeline.NDVI_CONFIG,
        feature_collection_pipeline.ERA5_CONFIG,
    ],
)


2022-05-18 00:22:16.855 | INFO     | src.data_processing.feature_collection_pipeline:collect_features_for_locations:86 - Authenticating with GEE...
2022-05-18 00:22:16.856 | DEBUG    | src.data_processing.gee.gee_utils:gee_auth:16 - /home/alron/unicef-ai4d-air-quality/data/unicef-ai4d-f1e103cbaeea.json
2022-05-18 00:22:20.104 | INFO     | src.data_processing.feature_collection_pipeline:collect_features_for_locations:90 - Computing population sums...
2022-05-18 00:22:22.737 | INFO     | src.data_processing.feature_collection_pipeline:collect_features_for_locations:96 - Collecting GEE datasets...
2022-05-18 00:22:22.738 | INFO     | src.data_processing.feature_collection_pipeline:collect_gee_datasets:152 - Collecting GEE data (1 / 4): {'collection_id': 'COPERNICUS/S5P/OFFL/L3_AER_AI', 'bands': ['absorbing_aerosol_index'], 'preprocessors': [<function aggregate_daily_s5p_aerosol at 0x7f89db4d3440>]}


{'collection_id': 'COPERNICUS/S5P/OFFL/L3_AER_AI', 'bands': ['absorbing_aerosol_index'], 'preprocessors': [<function aggregate_daily_s5p_aerosol at 0x7f89db4d3440>]}


  0%|          | 0/206 [00:00<?, ?it/s]

2022-05-18 00:24:15.469 | INFO     | src.data_processing.feature_collection_pipeline:collect_gee_datasets:152 - Collecting GEE data (2 / 4): {'collection_id': 'ECMWF/CAMS/NRT', 'bands': ['total_aerosol_optical_depth_at_550nm_surface'], 'preprocessors': [<function rescale_cams_aod at 0x7f89db4d3320>, <function aggregate_daily_cams_aod at 0x7f89db4d33b0>]}


{'collection_id': 'ECMWF/CAMS/NRT', 'bands': ['total_aerosol_optical_depth_at_550nm_surface'], 'preprocessors': [<function rescale_cams_aod at 0x7f89db4d3320>, <function aggregate_daily_cams_aod at 0x7f89db4d33b0>]}


...

{'collection_id': 'MODIS/006/MOD13A2', 'bands': ['NDVI', 'EVI'], 'preprocessors': [<function aggregate_daily_ndvi at 0x7f89db4d3b00>]}


  0%|          | 0/206 [00:00<?, ?it/s]

2022-05-18 00:27:34.517 | INFO     | src.data_processing.feature_collection_pipeline:collect_gee_datasets:152 - Collecting GEE data (4 / 4): {'collection_id': 'ECMWF/ERA5_LAND/HOURLY', 'bands': ['dewpoint_temperature_2m', 'temperature_2m', 'total_precipitation_hourly', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'surface_pressure'], 'preprocessors': [<function aggregate_daily_era5 at 0x7f89db4d3290>]}


{'collection_id': 'ECMWF/ERA5_LAND/HOURLY', 'bands': ['dewpoint_temperature_2m', 'temperature_2m', 'total_precipitation_hourly', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'surface_pressure'], 'preprocessors': [<function aggregate_daily_era5 at 0x7f89db4d3290>]}


  0%|          | 0/206 [00:00<?, ?it/s]

In [167]:
# Load Model
model = joblib.load(MODEL_PATH)

# Filter to only the relevant columns
keep_cols = model.feature_names  # This was saved from the train script
ml_df = base_df[keep_cols]

# Run model
preds = model.predict(ml_df)
base_df["predicted_pm2.5"] = preds
base_df.to_csv(OUTPUT_PREDICTIONS_FILE, index=False)


# Quick checks on predictions

In [173]:
results_df = pd.read_csv(OUTPUT_PREDICTIONS_FILE)
results_df = gpd.GeoDataFrame(results_df, geometry=gpd.points_from_xy(results_df.longitude, results_df.latitude))

In [176]:

print(f"Min:{min(preds):.2f} - Max:{max(preds):.2f}; Mean={np.mean(preds):.2f}")

Min:24.05 - Max:165.48; Mean=81.61
